In [54]:
import pandas as pd
from sodapy import Socrata
import nltk
import requests
import re

In [2]:
###GIVES THE COUNT OF `word' in 'text`

def matches(text):
    return sum(word in text.lower() for word in keywords)

In [ ]:
# ###GET DSLD LABEL DATA HERE:

# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# client = Socrata("datadiscovery.nlm.nih.gov", None)

# results = client.get("wp6t-qxsk", limit=89458)

# # Convert to pandas DataFrame
# dsld_lbl_df = pd.DataFrame.from_records(results)

In [76]:
# ###GET DSLD INGREDIENTS DATA HERE:

# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# client = Socrata("datadiscovery.nlm.nih.gov", None)

# results = client.get("btg5-fr69", limit=42360)

# # Convert to pandas DataFrame
# dsld_grdnt_df = pd.DataFrame.from_records(results)

In [104]:
###TAKE THIS INFO FROM AMAZON:
#here is an example: 
#https://www.amazon.com/Enzymedica-Potency-Enzymes-Digestive-Capsules/dp/B00HLXQ0HG/ref=sr_1_5?dchild=1&keywords=enzymedica+reducer&qid=1611441007&s=hpc&sr=1-5

#sentence is the name of the product from the amazon page
sentence = "Enzymedica, Digest Gold + ATPro, Digestive Enzymes, 180 Capsules (FFP)"

#regex_brand is the name of the product from the amazon page
brand = "Enzymedica"

In [105]:
#Query DSLD products by amazon's brand name:

query = re.sub("\s+", "%", "'"+'%'+brand+'%'+"'") 

response = requests.get("https://datadiscovery.nlm.nih.gov/resource/wp6t-qxsk.json?$where=brand_name LIKE "+query).json()

dsld_id = []
brand_name = []
product_name = []

for x in response:
    dsld_id.append(x['dsld_id'])
    brand_name.append(x['brand_name'])
    product_name.append(x['product_name'])
    
dsld_lbl_df = pd.DataFrame() # create empty dataframe 
dsld_lbl_df['dsld_id'] = dsld_id 
dsld_lbl_df['brand_name'] = brand_name
dsld_lbl_df['product_name'] = product_name ###add new columns based off lists

del dsld_id, brand_name, product_name
dsld_lbl_df

,dsld_id,brand_name,product_name
0,3018,Enzymedica,Enzymedica Acid Soothe
1,32413,Enzymedica,Enzymedica Acid Soothe
2,32419,Enzymedica,Enzymedica Acid Soothe
3,32420,Enzymedica,Enzymedica Allerase
4,30776,Enzymedica,Enzymedica BeanAssist
5,32424,Enzymedica,Enzymedica Candidase
6,32441,Enzymedica,Enzymedica Candidase
7,32445,Enzymedica,Enzymedica CarbGest
8,32452,Enzymedica,Enzymedica Digest
9,32461,Enzymedica,Enzymedica Digest


In [122]:
###GET DSLD ID HERE:

tokens = nltk.word_tokenize(sentence)
texts = [str(i) for i in dsld_lbl_df.product_name.tolist()]

keywords = [i.lower() for i in tokens]

regex_name = max(texts, key=matches)

#here, if I have several matches, I just take the one on the top of the list
dsld_id = dsld_lbl_df[
    (dsld_lbl_df["product_name"] == regex_name)
#     & 
#     dsld_lbl_df["brand_name"].str.contains(brand, regex=True, case=False).fillna(False)
].head(n=1).dsld_id.values[0]
dsld_id

'32494'

In [125]:
#Query DSLD ingredients by amazon's brand name:
query = "'"+'%'+dsld_id+'%'+"'"
response = requests.get("https://datadiscovery.nlm.nih.gov/resource/btg5-fr69.json?$where=sample_dsld_ids LIKE "+query).json()

ingredients = []

for product in response:
    ingredients.append(product['ingredient_name'])
    
# del dsld_id, brand_name, product_name
ingredients

['L. SALVARIUS', 'DIGEST GOLD BLEND', 'L. BULGARIUS']